In [2]:
import pandas as pd
import os

In [3]:
dir = '../Clean_Data/Teams'

for team in os.listdir(dir):
    offense_dir = os.path.join(dir, team, "Offense", "plays.csv")
    defense_dir = os.path.join(dir, team, "Defense", "plays.csv")

    offense_df = pd.read_csv(offense_dir)
    defense_df = pd.read_csv(defense_dir)

    if(len(offense_df['OffenseTeam'].unique())) > 1:
        print(team)
    if(len(defense_df['DefenseTeam'].unique())) > 1:
        print(team)



In [4]:
def get_week(dates):
    week_numbers = {}
    week_num = 1

    # Initialize week_start to the Thursday before the first game
    week_start = dates[0] - pd.Timedelta(days=(dates[0].weekday() + 4) % 7)

    for date in dates:
        # Check if we need to start a new week
        if date - week_start >= pd.Timedelta(days=7):  
            week_num += 1
            week_start = date - pd.Timedelta(days=(date.weekday() + 4) % 7)  # Adjust to the new Thursday

        date_str = date.strftime('%Y-%m-%d')
        week_numbers[date_str] = week_num

    return week_numbers
    

In [5]:
df = pd.read_csv('../Data/pbp-2023.csv')

dates = (df['GameDate'].unique())
dates = pd.to_datetime(dates)
dates = sorted(dates)

dates_with_weeks = get_week(dates)

In [7]:
import re

def extract_spot(play_description):
    # Regex to capture the team acronym and the spot
    match = re.search(r'ENFORCED AT (\w+ \d+)', play_description)
    if match:
        # Extract the spot part
        spot = match.group(1).split()
        return spot
    else:
        return 0

# Example usage
play_description = "10-I.PACHECO UP THE MIDDLE TO KC 29 FOR 6 YARDS (2-R.JENKINS). PENALTY ON KC-74-J.TAYLOR, OFFENSIVE HOLDING, 10 YARDS, ENFORCED AT KC 26."
spot = extract_spot(play_description)
print(spot)  # Should print 26


['KC', '26']


In [8]:
print(df['PlayType'].unique())

['RUSH' 'CLOCK STOP' 'FIELD GOAL' nan 'KICK OFF' 'PASS' 'SACK' 'TIMEOUT'
 'SCRAMBLE' 'NO PLAY' 'TWO-POINT CONVERSION' 'EXTRA POINT' 'QB KNEEL'
 'FUMBLES' 'PUNT' 'EXCEPTION']


In [9]:
dal_df = pd.read_csv('../data.csv')
print(dal_df.iloc[1]['Week']['PassY']['RushY'])

print(dal_df.columns)



IndexError: invalid index to scalar variable.

In [12]:
for _, row in dal_df.iterrows():
    print(row['Week'] , " : ", row['PassY'], row['RushY'])

1  :  143.0 122.0
2  :  248.0 134.0
3  :  231.0 185.0
4  :  253.0 124.0
5  :  140.0 57.0
6  :  246.0 96.0
7  :  nan nan
8  :  285.0 102.0
9  :  333.0 73.0
10  :  472.0 168.0
11  :  204.0 107.0
12  :  331.0 100.0
13  :  275.0 136.0
14  :  256.0 138.0
15  :  106.0 89.0
16  :  242.0 97.0
17  :  323.0 61.0
18  :  309.0 131.0


In [32]:
def extract_spot(play_description):
    # Regex to capture the team acronym and the spot
    match = re.search(r'ENFORCED AT (\w+ \d+)', play_description)
    if match:
        # Extract the spot part
        spot = match.group(1).split()
        return spot
    else:
        return 0

def rush_ended_at(play):
    match = re.search(r'FOR (-?\d+) YARDS', play['Description'])
    print(play['Description'])
    print(match.group(1))
    if match:
        spot = match.group(1)
       
        return int(spot)
        
# Rushing yards can have holding calls enforced past the LOS and that difference counts for a rush & yards
def rush_enforced_at_fix(play):
    spot = extract_spot(play['Description'])

    team_yard_line = spot[0]
    yard_line = spot[1]
    if play['OffenseTeam'] == team_yard_line:
        return int(yard_line) - int(play['YardLineFixed'])

    return int(play['YardLineFixed'] - int(yard_line))

def kneel_yards(play):
    match = re.search(r'FOR (-?\d+) YARDS', play['Description'])

    if match:
        return int(match.group(1))
    else:
        # kneels for no-gain
        return int(0)

def check_rush(plays, rush_stats, play):
    if play['PlayType'] == 'QB KNEEL':
        rush_stats['plays'] += 1
        rush_stats['attempts'] += 1

        yards = kneel_yards(play)
        rush_stats['yards'] += yards

        return rush_stats

    if play['IsNoPlay'] == 0 and (play['PenaltyTeam'] != play['OffenseTeam'] or play['IsPenaltyAccepted'] == 0) :
        rush_stats['plays'] += 1
        rush_stats['attempts'] += 1

        if play['IsFumble'] == 1:
            rush_yards = rush_ended_at(play)
            rush_stats['yards'] += rush_yards
            rush_stats['fumbles'] += 1
            return rush_stats
        
        rush_stats['yards'] += play['Yards']
        
        if play['IsTouchdown'] == 1:
            rush_stats['td'] += 1
        if play['IsFumble'] == 1:
            rush_stats['fumbles'] += 1


    elif (play['PenaltyTeam'] == play['OffenseTeam'] and play['IsPenaltyAccepted'] == 1 and play['IsNoPlay'] == 0):
        try:
            gain = rush_enforced_at_fix(play)
            rush_stats['plays'] += 1
            rush_stats['attempts'] += 1
            rush_stats['yards'] += gain
            play['Yards'] = rush_stats['yards']
        

        except Exception as e:  
            print(play['Description'])
            print(e)

    return rush_stats

def check_pass(pass_stats, play):
    if play['IsNoPlay'] == 0 and play['IsInterception'] == 0 and play['IsSack'] == 0 and play['IsTwoPointConversion'] == 0:
        pass_stats['plays'] += 1
        pass_stats['attempts'] += 1
        if play['IsIncomplete'] == 0:
            pass_stats['completions'] += 1
            pass_stats['yards'] += play['Yards']
        
        if play['IsTouchdown'] == 1:
            pass_stats['td'] += 1
        
    if play['IsSack'] == 1:
        pass_stats['yards'] += play['Yards']

    if play['IsNoPlay'] == 0 and play['IsInterception'] == 1:
        pass_stats['attempts'] += 1
        pass_stats['int'] += 1
    
    return pass_stats


In [14]:
team_directory = "../Clean_Data/Teams"
team = 'DAL'
pd.set_option('display.max_colwidth', None)  # Ensure full display of 'Description'

offense_directory = os.path.join(team_directory, team, "Offense", "plays.csv")

df = pd.read_csv(offense_directory)

df_filtered = df[(df['Week'] == 9) & (df['Quarter'] == 2) & (df['Minute'] == 13)]


df_filtered.to_csv("output.csv", index=False)


In [58]:
def get_team_totals(team_directory = "../Clean_Data/Teams", team = "DAL") :     

    offense_directory = os.path.join(team_directory, team, "Offense", "plays.csv")

    df = pd.read_csv(offense_directory)

    offense_directory = os.path.join(team_directory, team, "Offense", "plays.csv")

    df = pd.read_csv(offense_directory)

    weeks = df['Week'].unique()
    fumble_plays = df[df['IsFumble'] == 1]
    print('Shape:', fumble_plays.shape)
    
    weeks = sorted(map(int, weeks))
    run_plays = []
    for i in range(1, 19) :
        week_data = df[df['Week'] == i]
        
        # Initialize weekly stats
        weekly_pass_stats = {
            "plays" : 0,
            "attempts" : 0,
            "completions" : 0,
            "yards" : 0,
            "td" : 0,
            "int" : 0,
        }

        weekly_rush_stats = {
            "plays" : 0,
            "attempts" : 0,
            "yards" : 0,
            "td" : 0,
            "fumbles" : 0 
        }
        
         
        count = 0
        for _, play in week_data.iterrows():
            if play['IsPass'] == 1 or play['PlayType'] == 'SACK':
                
                weekly_pass_stats = check_pass(weekly_pass_stats, play)
                
            elif play['IsRush'] == 1 or play['PlayType'] == 'QB KNEEL':
                if play['IsFumble'] == 1:
                    print('Here:')
                    run_plays.append(play)
                weekly_rush_stats = check_rush(run_plays, weekly_rush_stats, play)
                

        my_rush_yards = weekly_rush_stats['yards']
        
        rush_yards = dal_df[dal_df['Week'] == i]['RushY']

        if not rush_yards.empty:
            if rush_yards.values[0] != my_rush_yards:
                print('week', i, my_rush_yards, rush_yards.values[0])    
    print('Run play', run_plays)
    run_df = pd.DataFrame(run_plays)
    run_df.to_csv("output.csv", index=False)

In [ ]:
fumble_plays.to_csv("output.csv")

In [229]:
print(df['PlayType'].unique())

['RUSH' 'CLOCK STOP' 'FIELD GOAL' nan 'KICK OFF' 'PASS' 'SACK' 'TIMEOUT'
 'SCRAMBLE' 'NO PLAY' 'TWO-POINT CONVERSION' 'EXTRA POINT' 'QB KNEEL'
 'FUMBLES' 'PUNT' 'EXCEPTION']


In [59]:
get_team_totals()

Shape: (13, 41)
Here:
(10:59) 9-K.TURPIN RIGHT END TO NE 22 FOR 46 YARDS (27-M.BRYANT). FUMBLES (27-M.BRYANT), RECOVERED BY DAL-84-S.MCKEON AT NE 14.
46
Here:
(:47) 20-T.POLLARD LEFT END TO DAL 9 FOR 4 YARDS (54-F.WARNER). FUMBLES (54-F.WARNER), TOUCHED AT DAL 9, RECOVERED BY SF-90-K.GIVENS AT DAL 16. 90-K.GIVENS RAN OB AT DAL 16 FOR NO GAIN. THE REPLAY OFFICIAL REVIEWED THE LOOSE BALL RECOVERY RULING, AND THE PLAY WAS UPHELD. THE RULING ON THE FIELD STANDS.
4
week 7 0 nan
week 9 66 73.0
week 14 137 138.0
Run play [GameId                                                                                                                                                 2023100111
GameDate                                                                                                                                               2023-10-01
Quarter                                                                                                                                                    